In [1]:
with open(r'C:\Users\user\Desktop\NLP\reviews_data.txt',encoding="utf-8",errors="ignore") as f:
    data = f.read()

In [2]:
data[0:1000]

"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Bei

In [13]:
from nltk.tokenize import word_tokenize  
from nltk.tokenize import RegexpTokenizer
tokens = data.lower()  #convert to lowercase
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(tokens) #tokenize and remove punctuation
#sample doc1 tokenized
print(tokens[0:100])

['oct', '12', '2009', 'nice', 'trendy', 'hotel', 'location', 'not', 'too', 'bad', 'i', 'stayed', 'in', 'this', 'hotel', 'for', 'one', 'night', 'as', 'this', 'is', 'a', 'fairly', 'new', 'place', 'some', 'of', 'the', 'taxi', 'drivers', 'did', 'not', 'know', 'where', 'it', 'was', 'and', 'or', 'did', 'not', 'want', 'to', 'drive', 'there', 'once', 'i', 'have', 'eventually', 'arrived', 'at', 'the', 'hotel', 'i', 'was', 'very', 'pleasantly', 'surprised', 'with', 'the', 'decor', 'of', 'the', 'lobby', 'ground', 'floor', 'area', 'it', 'was', 'very', 'stylish', 'and', 'modern', 'i', 'found', 'the', 'reception', 's', 'staff', 'geeting', 'me', 'with', 'aloha', 'a', 'bit', 'out', 'of', 'place', 'but', 'i', 'guess', 'they', 'are', 'briefed', 'to', 'say', 'that', 'to', 'keep', 'up', 'the']


In [14]:
from nltk.corpus import stopwords
import re, string
def preprocess(list):
    l=[]
    for word in list:
        if word not in set(stopwords.words("english")) and len(word)>2:
            lem = WordNetLemmatizer()
            word=lem.lemmatize(word)
            word=re.sub(r"\d+","",word)
            word=word.strip()
            l.append(word)
        else:
            pass
    return l

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
clean= preprocess(tokens)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
print(clean[0:100])

In [12]:
import gensim
from gensim import models
model = gensim.models.Word2Vec(texts, min_count=10,size=100)
model.most_similar(positive=['france',"england"], negative=["australia"],topn=5)

NameError: name 'texts' is not defined